In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("주식분석")

LangSmith 추적을 시작합니다.
[프로젝트명]
주식분석


In [3]:
# 필요 라이브러리 임포트
import os
import json
import re
from pydantic import BaseModel, Field
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool

In [4]:
######## states 정의 ########
class InputState(TypedDict):
    start_input: str

class OverallState(TypedDict):
    user_input: str
    messages: Annotated[list, add_messages]
    Stock_Value_dict: dict

In [5]:
# 사용자 입력 노드 정의
def user_input_node(state: InputState):
    print("================================= calculation stock =================================")
    print("주식 가치를 분석합니다. 궁금하신 주식명을 말씀해주세요.")
    user_input = input("User: ")

    return {
        "user_input": user_input,
        "messages": [("user", user_input)],
        "Stock_Value_dict": {},  # 나중에 여기에 데이터 채워짐
    }

In [6]:
def search_query_generation_node(state: dict) -> dict:
    user_input = state["user_input"]
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 금융 정보를 찾기 위한 검색 쿼리를 생성하는 전문가입니다."),
        ("human", '''
사용자 입력: "{user_input}"을 참고하여 다음 항목들에 대해 검색 쿼리를 만들어주세요.

- 당기순이익
- 발행주식수
- 현재 주가
- 자본총계
- 자유현금흐름
- 영업이익
- 가중평균자본비용
- 예상미래현금흐름
- 성장률
- 주당배당금
- 기타 수익 관련 정보

출력 형식은 반드시 JSON 배열입니다. 예:
[
  "삼성전자 연결 기준 당기순이익",
  "삼성전자 발행주식수",
  ...
]''')
    ])

    messages = prompt.format_messages(user_input=user_input)
    response = llm.invoke(messages)

    try:
        cleaned = re.sub(r"^```(?:json)?\n|\n```$", "", response.content.strip())
        query_list = json.loads(cleaned)
    except Exception as e:
        print(f"❗ 검색 쿼리 파싱 실패: {e}")
        query_list = []

    return {
        **state,
        "query_list": query_list
    }

In [7]:
def query_refine_node(state: dict) -> dict:
    original_queries = state.get("query_list", [])
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

    refine_prompt = ChatPromptTemplate.from_messages([
        ("system", """
당신은 전자공시시스템(DART) 문서에 적합한 검색어를 보정하는 전문가입니다.

다음 원본 검색어 목록을 참고하여 DART 문서에 더 잘 매칭되도록 쿼리를 보정해주세요.
- 연결 기준, 개별 기준 등은 유지
- 연도는 포함하지 마세요
- 보정 후 출력은 반드시 JSON 배열입니다.
"""),
        ("human", """
원본 쿼리 목록:
{query_list}
""")
    ])

    messages = refine_prompt.format_messages(query_list=json.dumps(original_queries, ensure_ascii=False))
    response = llm.invoke(messages)

    try:
        cleaned = re.sub(r"^```(?:json)?\n|\n```$", "", response.content.strip())
        refined_queries = json.loads(cleaned)
    except Exception as e:
        print(f"❗ 쿼리 보정 실패: {e}")
        refined_queries = original_queries

    print("\n🔁 쿼리 보정 결과:")
    for o, r in zip(original_queries, refined_queries):
        print(f"- {o} → {r}")

    return {
        **state,
        "query_list": refined_queries,
        "previous_query": original_queries
    }

In [8]:
######## nodes.py ########
# --- 문서 로드 및 전처리 ---
loader = PyMuPDFLoader("stock_report/[삼성전자]분기보고서(2024.11.14).pdf")
docs = loader.load()

## : 문서 분할(Split Documents) <-----------추후 문서 제목 단위 분할로 변경 필요
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
split_documents = text_splitter.split_documents(docs)

# 중복 제거
unique_documents = []
seen_contents = set()

for doc in split_documents:
    content = doc.page_content.strip()
    if content not in seen_contents:
        seen_contents.add(content)
        unique_documents.append(doc)

print(f"원본 문서 수: {len(split_documents)}")
print(f"중복 제거 후 문서 수: {len(unique_documents)}")

## 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 벡터스토어 생성
vectorstore = Chroma.from_documents(documents=split_documents, embedding=embeddings, persist_directory="stock_report/chroma_db")

# 5. 검색기(Retriever) 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

원본 문서 수: 404
중복 제거 후 문서 수: 403


In [9]:
# 키 매핑 테이블 (한글 → 내부키)
key_mapping = {
    "당기순이익": "Net_Income",
    "발행주식수": "Shares_Outstanding",
    "현재 주가": "Stock_Price",
    "자본총계": "Shareholders_equity",
    "자유현금흐름": "Free_cash_flow",
    "영업이익": "Operating_income",
    "가중평균자본비용": "WACC",
    "예상미래현금흐름": "Projected_future_cash_flows",
    "성장률": "Growth_Rate",
    "주당배당금": "Dividend_per_share",
    "기타 수익 관련 정보": "Other_return_related_information",
}

summary_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 요약 프롬프트
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", """
당신은 기업의 재무 정보를 요약하는 분석 전문가입니다.
다음 문서를 읽고, 아래 항목에 대해 **정확한 값과 단위**를 JSON 형식으로 요약해 주세요.

반드시 아래 형식으로 응답하세요 (JSON):
{{
  "당기순이익": "...",
  "발행주식수": "...",
  "현재 주가": "...",
  "자본총계": "...",
  "자유현금흐름": "...",
  "영업이익": "...",
  "가중평균자본비용(WACC)": "...",
  "예상미래현금흐름": "...",
  "성장률": "...",
  "주당배당금": "...",
  "기타 수익 관련 정보": "..."
}}

값을 찾을 수 없거나 문서에 명확히 없으면 "없음"이라고 표기하세요.
"""),
    ("human", "{context}")
])

# 문서 검색
def retrieve_docs(query: str, retriever, k=5) -> List[str]:
    return list({doc.page_content.strip() for doc in retriever.invoke(query)})[:k]

# 컨텍스트 정리
def clean_context(docs: List[str]) -> str:
    context = "\n\n".join(docs)
    return context[:3000]  # 길이 제한

# 요약 실행
def summarize_context(context: str, llm) -> dict:
    messages = summary_prompt.format_messages(context=context)
    response = llm.invoke(messages)
    try:
        raw = re.sub(r"^```(?:json)?\\n|\\n```$", "", response.content.strip())
        parsed = json.loads(raw)
        return {key_mapping.get(k, k): v for k, v in parsed.items()}
    except Exception as e:
        print(f"❗ 요약 파싱 실패: {e}")
        return {}

# 병합
def merge(parsed: dict, stock_data: dict):
    for k, v in parsed.items():
        if k not in stock_data or stock_data[k] == "없음":
            stock_data[k] = v

# 전체 노드

def rag_query_node(state: dict) -> dict:
    print("\n================ RAG 검색 및 요약 =================")

    query_list = state.get("query_list", [])
    stock_data = {}

    for query in query_list:
        print(f"\n🔍 검색 쿼리: {query}")
        docs = retrieve_docs(query, retriever)
        context = clean_context(docs)
        parsed = summarize_context(context, summary_model)

        print("\n📄 context:")
        print(context[:500] + ("..." if len(context) > 500 else ""))

        print("\n🧠 요약 결과:")
        print(json.dumps(parsed, indent=2, ensure_ascii=False))

        merge(parsed, stock_data)

    # 누락된 항목 "없음" 채우기
    for key in key_mapping.values():
        if key not in stock_data:
            stock_data[key] = "없음"

    return {
        **state,
        "Stock_Value_dict": stock_data,
        "messages": state["messages"] + [AIMessage(content="📊 재무 요약 완료")]
    }


In [10]:
test_queries = [
    "삼성전자 연결 기준 당기순이익",
    "삼성전자 발행주식수",
    "삼성전자 현재 주가",
    "삼성전자 자본총계",
    "삼성전자 자유현금흐름",
    "삼성전자 영업이익",
    "삼성전자 가중평균자본비용",
    "삼성전자 예상 미래 현금흐름",
    "삼성전자 성장률",
    "삼성전자 주당 배당금",
    "삼성전자 기타 수익 관련 정보"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n🔍 ({i}) 검색 쿼리: {query}")

    docs = retriever.invoke(query)
    context = "\n\n".join({doc.page_content.strip() for doc in docs})[:3000]
    print(f"📄 Context 길이: {len(context)}")

    if not context.strip():
        print("❗ 관련 문서 없음.")
        continue

    messages = summary_prompt.format_messages(context=context)
    response = summary_model.invoke(messages)

    print(f"🧠 요약 결과:\n{response.content.strip()}")



🔍 (1) 검색 쿼리: 삼성전자 연결 기준 당기순이익
📄 Context 길이: 2214
🧠 요약 결과:
```json
{
  "당기순이익": "(2,475)",
  "발행주식수": "6,792,669,250주",
  "현재 주가": "없음",
  "자본총계": "679,267백만원",
  "자유현금흐름": "없음",
  "영업이익": "없음",
  "가중평균자본비용(WACC)": "없음",
  "예상미래현금흐름": "없음",
  "성장률": "없음",
  "주당배당금": "없음",
  "기타 수익 관련 정보": "없음"
}
```

🔍 (2) 검색 쿼리: 삼성전자 발행주식수
📄 Context 길이: 1967
🧠 요약 결과:
```json
{
  "당기순이익": "없음",
  "발행주식수": "없음",
  "현재 주가": "없음",
  "자본총계": "없음",
  "자유현금흐름": "없음",
  "영업이익": "없음",
  "가중평균자본비용(WACC)": "없음",
  "예상미래현금흐름": "없음",
  "성장률": "없음",
  "주당배당금": "없음",
  "기타 수익 관련 정보": "없음"
}
```

🔍 (3) 검색 쿼리: 삼성전자 현재 주가
📄 Context 길이: 2168
🧠 요약 결과:
```json
{
  "당기순이익": "없음",
  "발행주식수": "없음",
  "현재 주가": "없음",
  "자본총계": "없음",
  "자유현금흐름": "없음",
  "영업이익": "없음",
  "가중평균자본비용(WACC)": "없음",
  "예상미래현금흐름": "없음",
  "성장률": "없음",
  "주당배당금": "없음",
  "기타 수익 관련 정보": "없음"
}
```

🔍 (4) 검색 쿼리: 삼성전자 자본총계
📄 Context 길이: 2834
🧠 요약 결과:
```json
{
  "당기순이익": "없음",
  "발행주식수": "없음",
  "현재 주가": "없음",
  "자본총계": "12,235,974 백만원",
  "자유현금흐름": "없음",
